<a href="https://colab.research.google.com/github/afinn02/SI699-Corner-Kick-CV/blob/main/comp_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes:**
- We have been taking time to explore Roboflow and want to focus on the models with a high MAP (mean average precision - balances precision recall curve) & a high precision
- Why: prioritizing aversion of false positives so that we don't mistake the ref as a player or the shadow as a ball
- Model selection: Yolov5
- Use the pretrained dataset and use it for the maps and how things line up?
- Need for us to define some discrete questions: what are the relavent questions to ask? Basketball ex: sideline out of bounds plays would look at a metric like how far the person was from the defender. We need these discrete events to identify for ours. Distance between the person taking the corner -> label for short and long corners as well as the shape of the defenders. We will need 4 or 5 different aspects of this that we need to develop. Don't wait for the technical side we can count on the precise measurements of the distance between players. Want to automatically classify as short and long corners based on the distance there (k means, ntarp: run a cluster and breaks those two apart to define separation or shape combination. If we find one that is somewhat intuitive it will be a great achievement)
- This is a very specific parameter space since women ages 18-24 have different athltic abilities. Like in women's basketball vs mens basketball the importance of rebounds is different. If there is a finding that has been established here for professional mens leagues seeing if it holds true or not at high level collegiate dataset is an easy first win.
- check the literature on this
- Have theories of what makes a good setpiece -> need to think about it this way.

In [3]:
!git clone https://github.com/afinn02/SI699-Corner-Kick-CV.git

Cloning into 'SI699-Corner-Kick-CV'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [4]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [ ]:
import pandas as pd

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8s')

FileNotFoundError: 'yolov8' does not exist

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -U -r requirements.txt

# Install OpenCV, numpy, matplotlib for image processing and visualization
!pip install opencv-python numpy matplotlib

Cloning into 'yolov5'...
remote: Enumerating objects: 17270, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 17270 (delta 0), reused 0 (delta 0), pack-reused 17269 (from 2)
Receiving objects: 100% (17270/17270), 16.11 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (11861/11861), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

When we have film downloaded, put it in a google drive spot, access here...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# # Example: Dataset in Google Drive
dataset_path = '/content/drive/MyDrive/SI699_Project/Corner_Film/Wisconsin'

to define the output of the model...

In [ ]:
train: "/content/drive/MyDrive/soccer_dataset/train/"
val: "/content/drive/MyDrive/soccer_dataset/val/"
nc: 4  # Number of classes (players, ball, referee, etc.)
class_names: ["player", "ball", "referee", "goalkeeper"]

to train the yolov5 model...

In [ ]:
# Clone YOLOv5 repository (if not already done)
# !git clone https://github.com/ultralytics/yolov5.git
# %cd yolov5

# # Install dependencies
# !pip install -r requirements.txt

# Train the model
!python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/soccer_dataset/data.yaml --weights yolov5s.pt --cache


In [ ]:
# !python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/soccer_dataset/data.yaml --weights yolov5s.pt --cache


Once trained, can run model on new images with this...

In [ ]:
# Inference on a new image or video
# !python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.5 --source /content/drive/MyDrive/soccer_test_video.mp4
